# 7.5. Valoración de opciones financieras.

- Carga el fichero: under_9126_.txt
- Pon cada campo con el tipo de datos correspondiente.
- Crea dos dataframes, uno para las puts y otro para las calls.
- Tendrán las siguientes columnas: ['Fecha Expir', 'Strike', 'Call_Compra',] y ['Fecha Expir', 'Strike', 'Put_Compra',]]
- Elimina los nans.
- Realiza un plot para cada vencimiento del precio frente al strike.
- Utilizando la librería https://github.com/yassinemaaroufi/MibianLib/tree/master/mibian calcula la volatilidad implícita de cada una de las opciones. Utiliza los siguientes parámetos:
underlyingPrice = 9126
interestRate = 0.01
annualDividends = 4
- Realiza una figura de la superfice de volatilidad: días de vencimiento vs strike vs volatilidas implícita para las calls y las puts.

### Carga del fichero

In [ ]:
import pandas as pd
import matplotlib as plt
#%matplotlib inline

In [ ]:
file = 'under_9126_.txt'
data = pd.read_csv(file,sep='\t', decimal=",", thousands='.')
data = data.reset_index()
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data = data.drop(['Fecha', 'Hora', 'Nombre Subya'], axis=1)
data.columns = [
    'Call_CC', 'Call_Compra', 'Call_Venta', 'Call_CV', 
    'Call_Ultimo', 'Call_Contratos', 'Call_Delta',
    'Fecha Expir', 'Strike', 'Put_Delta', 
    'Put_Contratos', 'Put_Ultimo', 'Put_CC',
    'Put_Compra', 'Put_Venta', 'Put_CV'
]
num_col = [
    'Call_CC', 'Call_Compra', 'Call_Venta', 'Call_CV', 'Call_Ultimo',
    'Call_Contratos', 'Call_Delta', 'Strike', 'Put_Delta',
    'Put_Contratos', 'Put_Ultimo', 'Put_CC', 'Put_Compra', 'Put_Venta',
    'Put_CV'
]
def to_num_col(x):
    x = x.str.replace('.', '')
    x = x.str.replace(',', '.')
    return x
data.loc[:, num_col] = data[num_col].apply(to_num_col)
#data.set_index(['Fecha Expir', 'Strike'], inplace=True)
#data.set_index(['Fecha Expir'], inplace=True)
#data = data.loc[:, ['Fecha Expir', 'Strike', 'Call_Ultimo', 'Put_Ultimo']]
#data['Fecha Expir'] = pd.to_datetime(data['Fecha Expir'])
#data[['Strike', 'Call_Ultimo', 'Put_Ultimo']] = data[['Strike', 'Call_Ultimo', 'Put_Ultimo']].apply(pd.to_numeric)
data['Fecha Expir'] = pd.to_datetime(data['Fecha Expir'])
data[num_col] = data[num_col].apply(pd.to_numeric)

In [ ]:
data_call = data[['Fecha Expir', 'Strike', 'Call_Compra',]]
data_put = data[['Fecha Expir', 'Strike', 'Put_Compra',]]
data_call = data_call.dropna()
data_put = data_put.dropna()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 10))
for date, new_df in data_call.groupby('Fecha Expir'):
    new_df.plot(x='Strike', y='Call_Compra', ax=ax, label=date)
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
for date, new_df in data_put.groupby('Fecha Expir'):
    new_df.plot(x='Strike', y='Put_Compra', ax=ax, label=date)
ax.legend()

In [ ]:
# otra manera
data_mindex = data_call.set_index(['Fecha Expir', 'Strike'])
data_mindex['Call_Compra'].unstack(level=-1).T.plot() # layout=(1,2) subplots=True

In [ ]:
# otra manera
data_mindex = data_put.set_index(['Fecha Expir', 'Strike'])
data_mindex['Put_Compra'].unstack(level=-1).T.plot() # layout=(1,2) subplots=True

### Calulo de volatilidades:
- https://github.com/yassinemaaroufi/MibianLib/tree/master/mibian
- http://www.option-price.com/implied-volatility.php

In [ ]:
pip install mibian

In [ ]:
import mibian
import datetime

In [ ]:
underlyingPrice = 9126
interestRate = 0.01
annualDividends = 4

def compute_imp_vol_call(x):
    print(x)   
    #Me([underlyingPrice, strikePrice, interestRate, annualDividends, daysToExpiration], volatility=x, callPrice=y, putPrice=z)
    option_params = [
        underlyingPrice,
        x['Strike'],
        interestRate,
        annualDividends,
        x['daysToExpiration']
    ]
    obj_mibian = mibian.Me(option_params, callPrice=x['Call_Compra'])
    call_vol = obj_mibian.impliedVolatility
    x.at['call_vol'] = call_vol
    return x

def compute_imp_vol_put(x):
    print(x)   
    #Me([underlyingPrice, strikePrice, interestRate, annualDividends, daysToExpiration], volatility=x, callPrice=y, putPrice=z)
    option_params = [
        underlyingPrice,
        x['Strike'],
        interestRate,
        annualDividends,
        x['daysToExpiration']
    ]
    obj_mibian = mibian.Me(option_params, putPrice=x['Put_Compra'])
    put_vol = obj_mibian.impliedVolatility
    x.at['put_vol'] = put_vol
    return x

In [ ]:
import datetime
today = datetime.datetime.today()
data_put['daysToExpiration'] = (data_put['Fecha Expir'] - today).dt.days

In [ ]:
data_put = data_put.apply(compute_imp_vol_put, axis=1)

In [ ]:
data_put.head()

In [ ]:
data_call['daysToExpiration'] = (data_call['Fecha Expir'] - today).dt.days
data_call = data_call.apply(compute_imp_vol_call, axis=1)

- Ejemplo plot 3d
- Tutorial: https://jakevdp.github.io/PythonDataScienceHandbook/04.12-three-dimensional-plotting.html
- Superficie de volatilidad:

In [ ]:
calls_volatilit = data_call.set_index(['daysToExpiration', 'Strike'])['call_vol'].unstack(level=-1)
calls_volatilit = calls_volatilit.fillna(calls_volatilit.mean())

In [ ]:
x = calls_volatilit.index.values
y = calls_volatilit.columns.values
z = calls_volatilit.values

In [ ]:
import numpy as np
from mpl_toolkits import mplot3d

fig = plt.figure(figsize=(10, 7))
ax = plt.axes(projection='3d')
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, z.T)